In [4]:
from pathlib import Path
import skimage.io as io
from scipy.ndimage.interpolation import zoom as npzoom
from skimage.util import img_as_float32, img_as_ubyte
from skimage.measure import compare_ssim, compare_psnr
from fastprogress import progress_bar
import pandas as pd

movie_results = Path('/DATA/Dropbox/bpho_movie_results/')

In [7]:
def key(fn): 
    return '_'.join(fn.stem.split('_')[:-1])

emtru = {key(fn):fn for fn in (movie_results/'em_srgan').glob('*tru*')}
emorg = {key(fn):fn for fn in (movie_results/'em_srgan').glob('*orig*')}
emsrg = {key(fn):fn for fn in (movie_results/'em_srgan').glob('*pred*')}
emmse = {key(fn):fn for fn in (movie_results/'emsynth_mse_loss').glob('*pred*')}
empxl = {key(fn):fn for fn in (movie_results/'emsynth_003').glob('*pred*')}


models = dict(srgan=emsrg, mse_loss=emmse, pixel_loss=empxl)
stats = []

for k in progress_bar(emtru):
    oimg = img_as_float32(io.imread(emorg[k]))
    timg = img_as_float32(io.imread(emtru[k]))
    
    cimg = npzoom(oimg, 4)
    stats.append({'model': 'bicubic', 'metric': 'ssim', 'value': compare_ssim(timg, cimg)})
    stats.append({'model': 'bicubic', 'metric': 'psnr', 'value': compare_psnr(timg, cimg)})

    limg = npzoom(oimg, 4, order=1)
    stats.append({'model': 'bilinear', 'metric': 'ssim', 'value': compare_ssim(timg, limg)})
    stats.append({'model': 'bilinear', 'metric': 'psnr', 'value': compare_psnr(timg, limg)})
     
    model_imgs = {}
    for lbl, fnmap in models.items():
        pimg = img_as_float32(io.imread(fnmap[k]))
        if len(pimg.shape) == 3: pimg = pimg[:,:,0]
        
        stats.append({'model': lbl, 'metric': 'ssim', 'value': compare_ssim(timg, pimg)})
        stats.append({'model': lbl, 'metric': 'psnr', 'value': compare_psnr(timg, pimg)})

In [10]:
df = pd.DataFrame(stats)

In [11]:
df.head()

,metric,model,value
0,ssim,bicubic,0.694413
1,psnr,bicubic,18.622222
2,ssim,bilinear,0.731585
3,psnr,bilinear,18.784716
4,ssim,srgan,0.461154


In [19]:
df.groupby(['metric','model']).describe()

value                                                        \
                  count       mean       std        min        25%        50%   
metric model                                                                    
psnr   bicubic     85.0  19.892473  3.397313  14.207584  16.520311  19.531696   
       bilinear    85.0  20.155426  3.544903  14.275473  16.623836  19.738787   
       mse_loss    85.0  20.296502  3.607377  14.356248  16.690579  19.862071   
       pixel_loss  85.0  20.144021  3.425759  14.218888  16.777361  19.791488   
       srgan       85.0  17.972157  2.228699  13.677290  15.873811  17.945053   
ssim   bicubic     85.0   0.692300  0.036899   0.620273   0.662589   0.705555   
       bilinear    85.0   0.730810  0.036152   0.658729   0.702204   0.744095   
       mse_loss    85.0   0.760684  0.034061   0.688958   0.734282   0.772526   
       pixel_loss  85.0   0.740503  0.032418   0.672091   0.715363   0.750805   
       srgan       85.0   0.435129  0.015627   0.401550   0.425062   0.436049   

                                         
                         75%        max  
metric model                             
psnr   bicubic     23.139879  24.248150  
       bilinear    23.557128  24.759367  
       mse_loss    23.762687  25.014664  
       pixel_loss  23.435396  24.657583  
       srgan       20.026971  20.520869  
ssim   bicubic      0.724061   0.746718  
       bilinear     0.761960   0.785502  
       mse_loss     0.789989   0.815255  
       pixel_loss   0.768237   0.791780  
       srgan        0.444478   0.471554